# Energy profile vs. beam intensity

To do: check whether the mean x and y on the BPM is the same at each intensity.

In [ ]:
import os
from os.path import join
from collections import Counter
from datetime import datetime
import h5py
import numpy as np
from tqdm import trange
from scipy import optimize as opt
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib import cm
import proplot as pplt

pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'dusk_r'

In [ ]:
folder = 'Diagnostics/Data/Measurements/2022-04-27/'
filenames = os.listdir(folder)
filenames = sorted(filenames, key=lambda filename: int(filename.split('-')[0]))
filenames

In [ ]:
filename = filenames[0]
file = h5py.File(join(folder, filename), 'r')
list(file.keys())

In [ ]:
# Errors and warnings from log
for i in range(file['log'].size):
    if not(file['/log'][i, 'level'] == 'INFO'.encode('utf')):
        timestr = datetime.fromtimestamp(file['/log'][0, 'timestamp']).strftime("%m/%d/%Y, %H:%M:%S")
        print(f"{timestr} {file['log'][i, 'message']}")

# Configuration data
for key in file['/config'].keys():
    print(f"{key}")
    print("--------------")
    for name in file['/config'][key].dtype.names:
        print(f"{name}: {file['config'][key][name]}")
    print()

In [ ]:
data = file['scandata']
print('Data:')
for i, name in enumerate(data.dtype.names):
    print(f'{name} [{data.dtype[i]}]')
print()

slits = [key for key in data.dtype.names if 'PositionSync' in key]
print('slits:')
for slit in slits:
    print(slit)

In [ ]:
for name in data.dtype.names:
    if name in ['timestamp', 'cam06_Image', 'cam06_ProfileX']:
        continue
    fig, ax = pplt.subplots(figsize=(3.0, 2.5))
    ax.plot(data[name], color='black')
    ax.format(xlabel='step', ylabel=name)
    plt.show()

In [ ]:
fig, ax = pplt.subplots(figsize=(3.0, 2.5))
ax.format(xlabel='Step', ylabel='Current [mA]')
ax2 = ax.alty(color='grey')
ax2.format(ylabel='LEBT Focus1')
ax2.plot(data['focus1'], color='grey')
ax.plot(data['bcm'], color='black')
plt.savefig('_output/current')

In [ ]:
bpm_wfs = dict()
bpm_means = dict()
bpm_stds = dict()
bpm_keys = ['x', 'y', 'u', 'd', 'l', 'r', 'xraw', 'yraw']
for key in bpm_keys:
    bpm_wfs[key] = []
    bpm_means[key] = []
    bpm_stds[key] = []

In [ ]:
iterations = []
profiles = []
currents = []
images = []
lo, hi = 566, 595
for iteration in trange(1, max(data['iteration']) + 1):
    iterations.append(iteration)
    idx = data['iteration'] == iteration
    image = np.mean(data['cam06_Image'][idx, :], axis=0).reshape((512, 612))
    images.append(image)
    profile = np.mean(data['cam06_ProfileX'][idx, :], axis=0)  # averagine
    profile = profile / np.sum(profile)  # normalize
    profiles.append(profile)
    currents.append(data['bcm'][idx][0])
    # BPM waveforms
    cut = 999  # some waveforms are zero beyond this index
    bpm_wf = dict()
    bpm_wf['x'] = np.mean(data['bpm04_x'][idx, :cut], axis=0)
    bpm_wf['y'] = np.mean(data['bpm04_y'][idx, :cut], axis=0)
    bpm_wf['u'] = np.mean(data['bpm04_magu'][idx, :cut], axis=0)
    bpm_wf['d'] = np.mean(data['bpm04_magd'][idx, :cut], axis=0)
    bpm_wf['l'] = np.mean(data['bpm04_magl'][idx, :cut], axis=0)
    bpm_wf['r'] = np.mean(data['bpm04_magr'][idx, :cut], axis=0)
    bpm_wf['xraw'] = 8.0 * np.log10(bpm_wf['l'] / bpm_wf['r'])
    bpm_wf['yraw'] = 8.0 * np.log10(bpm_wf['u'] / bpm_wf['d'])
    for key, wf in bpm_wf.items():
        window = wf[lo:hi]
        bpm_means[key].append(np.mean(window))
        bpm_stds[key].append(np.std(window))
        bpm_wfs[key].append(wf)

In [ ]:
profiles = np.array(profiles)
currents = np.array(currents)
for key in bpm_wfs:
    bpm_wfs[key] = np.array(bpm_wfs[key])
    bpm_means[key] = np.array(bpm_means[key])
    bpm_stds[key] = np.array(bpm_stds[key])

The following cell creates a lot of plots.

In [ ]:
pad = 25
_lo = lo - pad
_hi = hi + pad
xx = np.arange(lo - pad, hi + pad)
# for i in range(len(profiles)):
#     fig, axes = pplt.subplots(ncols=8, figheight=1.5, sharey=False)
#     plot_kws = dict(color='black')
#     for ax, key in zip(axes, bpm_keys):
#         ax.plot(xx, bpm_wfs[key][i, _lo:_hi], **plot_kws)
#         ax.set_title(key)
#         ax.axvspan(lo, hi, color='black', alpha=0.05)
#     plt.show()

In [ ]:
cmap = pplt.Colormap('greys', left=0.12, right=0.9)
norm = colors.Normalize(np.min(np.abs(currents)), np.max(np.abs(currents)))
mappable = cm.ScalarMappable(norm, cmap)

In [ ]:
fig, axes = pplt.subplots(ncols=2, nrows=2, figsize=(5, 2.5), spany=False, aligny=True)
plot_kws = dict(lw=0.75)
for i in range(len(profiles)):
    color = mappable.to_rgba(np.abs(currents[i]))
    axes[0, 0].plot(xx, bpm_wfs['x'][i, _lo:_hi].T, color=color, **plot_kws)
    axes[1, 0].plot(xx, bpm_wfs['y'][i, _lo:_hi].T, color=color, **plot_kws)
    axes[0, 1].plot(xx, bpm_wfs['xraw'][i, _lo:_hi].T, color=color, **plot_kws)
    axes[1, 1].plot(xx, bpm_wfs['yraw'][i, _lo:_hi].T, color=color, **plot_kws)
for ax in axes:
    ax.axvspan(lo, hi, color='black', alpha=0.025)
fig.colorbar(mappable, label='Current [mA]')
axes[0, :].format(ylabel='x [mm]')
axes[1, :].format(ylabel='y [mm]')
axes.format(
    xlabel='Time', 
    toplabels=['From BPM electronics', 'From raw waveforms'], 
    toplabels_kw=dict(fontweight='normal'),
)
plt.savefig('_output/bpm')

In [ ]:
fig, ax = pplt.subplots()
plot_kws = dict(marker='.', ms=4.0, lw=0, elinewidth=1.0)
for key in ['x', 'y']:
    ax.errorbar(currents, bpm_means[key], yerr=bpm_stds[key], label=key, **plot_kws)
ax.format(cycle='colorblind')
plot_kws['alpha'] = 0.2
for key in ['xraw', 'yraw']:
    ax.errorbar(currents, bpm_means[key], yerr=bpm_stds[key], label=key, **plot_kws)
ax.format(xlabel='Current [mA]', ylabel='[mm]')
ax.legend(loc='right', ncols=1, framealpha=0.)
plt.savefig('_output/bpmavg')
plt.show()

In [ ]:
fig, ax = pplt.subplots(figsize=(4.0, 3.0))
for profile, current in zip(profiles, currents):
    ax.plot(profile, color = mappable.to_rgba(np.abs(current)))
ax.colorbar(mappable=mappable, label='Current [mA]')
ax.format(xlim=(100, ax.get_xlim()[1]), yticks=[], xlabel=r'~$\Delta$E', xticks=[], ylabel='Density')
plt.savefig('_output/hollow')

In [ ]:
fig, ax = pplt.subplots(
    figsize=(4.0, 3.0)
)
ax.plot(profiles.T, cycle=cmap)
ax.colorbar(mappable)
ax.format(xlim=(100, ax.get_xlim()[1]), yticks=[], xlabel=r'~$\Delta$E', xticks=[], ylabel='Density')
plt.savefig('_output/overlay')
plt.show()

In [ ]:
fig, ax = pplt.subplots(figsize=(5.0, 2.0))
ax.pcolormesh(
    np.arange(profiles.shape[1]),
    currents, 
    profiles, 
    cmap='mono_r', colorbar=True, colorbar_kw=dict(ticks=[]),
)
ax.format(ylabel='Current [mA]', xlabel=r'~$\Delta$E')
plt.savefig('waterfall_heatmap')
plt.show()

In [ ]:
fig, axes = pplt.subplots(nrows=4, ncols=5, figwidth=7.0, space=0.5)
for ax in axes[-2:]:
    ax.axis('off')
for image, ax, current in zip(images[1:], axes, currents):
    ax.imshow(image, cmap='mono_r')
    ax.annotate(f'I = {current:.2f}', xy=(0.01, 0.01), xycoords='axes fraction',
                color='white', fontsize='small',
                verticalalignment='bottom', horizontalalignment='left')
axes.format(xticks=[], yticks=[])
plt.savefig('_output/images')
plt.show()

In [ ]:
# import gif
# gif.options.matplotlib["dpi"] = 200

# @gif.frame
# def plot(i):
#     fig, ax = plt.subplots(figsize=(3.0, 3.0))
#     ax.imshow(images[i], cmap=pplt.Colormap('mono_r', left=0.04))
#     ax.annotate(f'I = {currents[i]:.2f} [mA]', xy=(0.01, 0.01), xycoords='axes fraction',
#                 color='white', fontsize='small',
#                 verticalalignment='bottom', horizontalalignment='left')
#     ax.set_xticks([])
#     ax.set_yticks([])
#     plt.savefig(f'_output/folder/image{i}.png')

# frames = [plot(i) for i in range(len(images))]
# gif.save(frames, '_output/images.gif', duration=0.25, unit="s", between="frames")